# Import dependencies

In [1]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Test enviroment

In [27]:
enviroment_name = 'LunarLander-v2'
env = gym.make(enviroment_name, render_mode='human')

In [11]:
env.reset()

(array([-0.00735188,  1.4166026 , -0.74468803,  0.25252444,  0.00852586,
         0.16868292,  0.        ,  0.        ], dtype=float32),
 {})

In [12]:
# 0: do nothing, 1: fire left orientation engine, 2: fire main engine, 3: fire right orientation engine
env.action_space

Discrete(4)

In [13]:
env.observation_space

Box([-90.        -90.         -5.         -5.         -3.1415927  -5.
  -0.         -0.       ], [90.        90.         5.         5.         3.1415927  5.
  1.         1.       ], (8,), float32)

In [14]:
env.render()

In [7]:
env.close()

In [15]:
episodes = 5
for episode in range(1, episodes+1):
  obs = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    score += reward

  print('Episode: {} Score {}'.format(episode, score))
#env.close()

Episode: 1 Score -358.81399274387405
Episode: 2 Score -386.89211047463016
Episode: 3 Score -139.7929743344552
Episode: 4 Score -301.87457266744883
Episode: 5 Score -199.50242978359483


# Train model

In [16]:
env = gym.make(enviroment_name, render_mode='human')
env = DummyVecEnv([lambda: env])

In [19]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [20]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_6
-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 43   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 45          |
|    iterations           | 2           |
|    time_elapsed         | 89          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007059358 |
|    clip_fraction        | 0.0152      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.00051     |
|    learning_rate        | 0.0003      |
|    loss                 | 676         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00594    |
|    value_loss           | 1.54e+03    |
-----------------------------------------
---

# Save model

In [21]:
Lunar_path = os.path.join('Training', 'Saved models', 'PPO_Lunar_Model')
model.save(Lunar_path)

In [28]:
del model

In [29]:
model = PPO.load(Lunar_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# Evaluate and test

In [22]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=True)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

c:\Users\william.mendes\Anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:-1223.65 +/- 276.43


In [23]:
env.close()

In [36]:
episodes = 5
for episode in range(1, episodes+1):
  obs = env.reset()[0]
  done = False
  score = 0

  while not done:
    env.render()
    action = model.predict(obs)[0]
    obs, reward, done, truncated, info = env.step(action)
    score += reward

  print('Episode: {} Score {}'.format(episode, score))
#env.close()

Episode: 1 Score -36.57091535522443
Episode: 2 Score -94.48805150454729
Episode: 3 Score -50.015125307593195
Episode: 4 Score 15.424553636535705
Episode: 5 Score -64.49155280427888
